In [1]:
from kp2d import KP2D
from kp2d.utils import *
import cv2
import os
import time
import shutil


In [2]:
def create_photo_list(image_directory):
    nms = []
    ext = []
    for file in os.scandir(image_directory):
        nms.append(file.name[:-4])
        ext.append(os.path.splitext(file.path)[1])
    if len(set(ext)) == 1:
        ext = ext[0]
    else:
        print('Ошибка! В папке не только картинки, или они разного формата.')
    photo_list = [name for name in nms]
    photo_list.sort(key=float)
    return photo_list, ext

def panorama(input_path, output_path, SIFT=False, rotate=False):

    start_time = time.time()

    if SIFT:
        descriptor = cv2.xfeatures2d.SIFT_create()
        reprojThresh = 4
        matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_FLANNBASED)

    else:
        use_gpu = True
        min_score = 0.7
        model_path = ".\models\keypoint_resnet.ckpt"
        reprojThresh = 4
        matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_FLANNBASED)


    
    photo_list, ext = create_photo_list(input_path)


    query_image = cv2.imread(
        os.path.join(input_path , photo_list[0]+ext))
    if rotate:
        query_image = cv2.rotate(query_image, cv2.ROTATE_90_CLOCKWISE)


    if SIFT:
        query_keypoints, query_features = descriptor.detectAndCompute(query_image, None)
        perimeter_ratio = 1

    else:
        input_size = (query_image.shape[1], query_image.shape[0])
        keypoint_detector = KP2D(model_path, input_size, min_score, use_gpu)
        query_scores, query_keypoints, query_features = keypoint_detector(query_image)
        perimeter_ratio = 1

    perimeter_x = query_image.shape[1] * perimeter_ratio
    perimeter_y = query_image.shape[0] * perimeter_ratio

    filtered_keypoints = [kp for kp in query_keypoints if (kp.pt[0] <= perimeter_x) or (kp.pt[0] >= query_image.shape[1] - perimeter_x) or
                        (kp.pt[1] <= perimeter_y) or (kp.pt[1] >= query_image.shape[0] - perimeter_y)]
    filtered_indices = [i for i, kp in enumerate(query_keypoints) if kp in filtered_keypoints]
    filtered_features = query_features[filtered_indices]

    homography_list = []
    corner_points_x = [0, query_image.shape[1]]
    corner_points_y = [0, query_image.shape[0]]

    keypoints_xy = np.float32([[kp.pt[0], kp.pt[1]] for kp in filtered_keypoints])

    for num_photo, photo in enumerate(photo_list[1:], start=1):
        train_image = cv2.imread(os.path.join(input_path , photo+ext))
        if rotate:
            train_image = cv2.rotate(train_image, cv2.ROTATE_90_CLOCKWISE)


        if SIFT:
            train_keypoints, train_features = descriptor.detectAndCompute(train_image, None)
        else:
            train_scores, train_keypoints, train_features = keypoint_detector(train_image)

        perimeter_x = train_image.shape[1] * perimeter_ratio
        perimeter_y = train_image.shape[0] * perimeter_ratio

        filtered_keypoints_2 = [kp for kp in train_keypoints if (kp.pt[0] <= perimeter_x) or (kp.pt[0] >= train_image.shape[1] - perimeter_x) or
                            (kp.pt[1] <= perimeter_y) or (kp.pt[1] >= train_image.shape[0] - perimeter_y)]
        filtered_indices_2 = [i for i, kp in enumerate(train_keypoints) if kp in filtered_keypoints_2]
        filtered_features_2 = train_features[filtered_indices_2]

        knn_matches = matcher.knnMatch(filtered_features_2, filtered_features, 2)
        

        ratio_thresh = 0.75
        good_matches = [m for m, n in knn_matches if m.distance < ratio_thresh * n.distance]

        obj = np.float32([filtered_keypoints_2[m.queryIdx].pt for m in good_matches])
        scene = keypoints_xy[[m.trainIdx for m in good_matches]]

        H, _ = cv2.findHomography(obj, scene, cv2.RANSAC, reprojThresh)
        homography_list.append(H)

        h, w, _ = train_image.shape
        corners_bef = np.float32([[0, 0], [w, 0], [w, h], [0, h]]).reshape(-1, 1, 2)
        corners_aft = cv2.perspectiveTransform(corners_bef, H)
        c = corners_aft.reshape(-1, 2)
        corner_points_x.extend([c[0, 0], c[1, 0], c[2, 0], c[3, 0]])
        corner_points_y.extend([c[0, 1], c[1, 1], c[2, 1], c[3, 1]])

        kps_first2 = np.float32([[kp.pt[0], kp.pt[1]] for kp in filtered_keypoints_2])
        corners_bef = kps_first2.reshape(-1, 1, 2)
        corners_aft = cv2.perspectiveTransform(corners_bef, H)
        c = corners_aft.reshape(-1, 2)
        keypoints_xy = np.vstack([keypoints_xy, c])
        filtered_features = np.vstack([filtered_features, filtered_features_2])

    time_1 = time.time() - start_time

    x_min = min(corner_points_x)
    x_max = max(corner_points_x)
    y_min = min(corner_points_y)
    y_max = max(corner_points_y)

    delta_x = int(x_max - x_min)
    delta_y = int(y_max - y_min)

    result = np.zeros((delta_y, delta_x, 3), dtype=np.uint8)
    result[0:query_image.shape[0], 0:query_image.shape[1]] = query_image

    for num_photo, photo in enumerate(photo_list[1:], start=1):
        train_image = cv2.imread(
            os.path.join(input_path , photo+ext))
        if rotate:
            train_image = cv2.rotate(train_image, cv2.ROTATE_90_CLOCKWISE)
        temp = cv2.warpPerspective(train_image, homography_list[num_photo - 1], (delta_x, delta_y), borderMode=cv2.BORDER_TRANSPARENT)
        nonzero_pixels = np.bitwise_and(np.any(temp, axis=2), np.bitwise_not(np.any(result, axis=2)))
        result[nonzero_pixels] = temp[nonzero_pixels]

    all_time = time.time() - start_time
    cv2.imwrite(os.path.join(output_path), result)
    print(all_time)
    print("Only the second loop: ", all_time - time_1)

Пример 1: панорама из видео

In [ ]:
# Сначала собираю кадры из видео, сохраняю и соединяю в панораму

for i, video in enumerate(os.scandir('examples\\vids\\vids')):
    print(video.path)
    save_best_frames_from_video(video.path, 'examples\\vids\\vid_img_out\\test', 8, 50)
    name = str(i).zfill(6)
    panorama('examples\\vids\\vid_img_out\\test', f'examples\\vids\\vid_img_out\\{name}.png',False)
    shutil.rmtree('examples\\vids\\vid_img_out\\test')

# Так как видео в довольно высоком разрешении и снято горизонтальными линиями,
# я поворачиваю и уменьшаю получившиеся кусочки 
for dirent in os.scandir('examples\\vids\\vid_img_out'):
    print(dirent.name)
    img = cv2.imread(dirent.path)
    img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    width = int(img.shape[1] * 50 / 100)
    height = int(img.shape[0] * 50 / 100)
    dim = (width, height)
    img = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
    cv2.imwrite(f'examples\\vid_img_out\\{dirent.name}', img)
panorama('examples\\vid_img_out', 'examples\\vids\\video_result.png')
# к сожалению, результат панорамы из видео получился недостаточно хорошим, но это следствие того, что неровно было снято видео
video_result = cv2.imread('examples\\vids\\video_result.png')
cv2.imshow('video result', video_result)
cv2.waitKey(0)
cv2.destroyAllWindows()

Пример 2: панорама из фото

In [55]:
# на ровно сфотографированной карте результат получается следующим
panorama('examples\img_input', 'examples\output\photo_res.png',False)
result = cv2.imread('examples\output\photo_res.png')
cv2.imshow('photo result', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

2.4846904277801514
Only the second loop:  0.906195878982544
